In [ ]:
# Extract tags using NLP
# filter them further using SPACY

from sklearn.feature_extraction.text import CountVectorizer
import spacy
nlp = spacy.load("en_core_web_sm")

def get_tags(desc):
    vectorizer = CountVectorizer(max_features=100, stop_words='english', ngram_range=(1, 2))
    X = vectorizer.fit_transform([desc])
    tags = vectorizer.get_feature_names_out()
    keep = []
    for term in tags:
        doc = nlp(term.replace("_", " "))
        if all(token.pos_ in ["NOUN", "ADJ"] for token in doc):
            keep.append(term.lower())
    return list(set(keep))

In [ ]:
# Extract tags using NLP
# filter them further using SPACY

from groq import Groq
import base64
import json

client = Groq(api_key=os.getenv("GROQ_API"))

map_ans = {
   "1": {"yes": ["formal"], "no": []},
   "2": {"yes": ["summer"], "no": []},
   "3": {"yes": ["date", "date-night", "romance", "cute", "beautiful"], "no": []},
   "4": {"yes": ["casual"], "no":[]},
   "5": {"yes": ["sexy", "hot", "attractive"], "no":[]},
   "6": {"yes": ["beach", "pool"], "no":[]},
   "7": {"yes": ["bikini", "swimwear", "swim", "ocean", "beach"], "no":[]},
   "8": {"yes": ["streetwear"], "no":["urban"]},
   "9": {"yes": ["sport", "workout", "gym", "fit"], "no":[]},
   "10": {"yes": ["party", "casual", "chill", "cool"], "no":[]},
   "11": {"yes": ["modest"], "no":["conservative"]},
   "12": {"yes": ["trendy"], "no":[]},
   "13": {"yes": ["retro", "vintage", "nostalgic"], "no":[]},
   "14": {"yes": ["winter"], "no":[]},
   "15": {"yes": ["layering"], "no":[]},
   "16": {"yes": ["light", "lightweight", "breathable", "breath", "cool"], "no":[]},
   "17": {"yes": ["business", "work", "office", "professional"], "no":[]},
   "18": {"yes": ["minimalist"], "no":["maximalist", "bold"]},
   "19": {"yes": ["luxury"], "no":[]},
   "20": {"yes": ["travel"], "no":["comfy"]},
}

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


def get_tags(img, name):
    completion = client.chat.completions.create(
        
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
      {
        "role": "system",
        "content": "Take name and photo of a product and answer 20 static questions about it. Your task is to analyze the product and determine the most suitable tags for each question.\n\n**Input:**\n\n* A single photo of a product (attachment)\n\n**Task:**\n\n1. Analyze the photo and determine the most suitable answer for each of the 20 questions.\n2. Based on your analysis, assign a tag to each question (e.g. \"yes\", \"no\", \"formal\", \"summer\", etc.)\n3. Return the answers as a JSON object with the question number as the key and the assigned tag as the value.\n\n**Questions:**\n\n1. Can this product be worn in a formal Setting? \n2. Can this product be worn in summer weather? \n3. Is this product suitable for a date night? \n4. Is this product good for casual Setting? \n5. Is this product considered Sexy? \n6. Is this item for the beach or poolside? \n7. Is this item from swimwear? is it a bikini? \n8. Does this product lean toward streetwear or urban fashion (yes for streetwear)? \n9. Is this suitable for working out or sports activities? \n10. Would you wear this to a party or club? \n11. Does this outfit appear modest or conservative (yes for modest)? \n12. Is this product aligned with current trends? \n13. Does this outfit have a nostalgic or vintage feel? \n14. Is this item suitable for cold weather and winter?\n15. Can this item be layered with other pieces (like jackets)? \n16. Is the fabric light and breathable? \n17. Would you wear this to work or a professional setting? \n18. Is the aesthetic minimalist or maximalist (yes for minimalist)? \n19. Does this product look luxurious or high-end? \n20. Is this item suitable for travel or long wear (yes for travel)? \n\n**JSON Output:**\n```json\n{\n  \"1\": \"no\",\n  \"2\": \"yes\",\n  \"3\": \"no\",\n  \"4\": \"no\",\n  \"5\": \"yes\",\n  \"6\": \"no\",\n  \"7\": \"yes\",\n  \"8\": \"yes\",\n  \"9\": \"no\",\n  \"10\": \"yes\",\n  \"11\": \"yes\",\n  \"12\": \"no\",\n  \"13\": \"yes\",\n  \"15\": \"yes\",\n  \"16\": \"yes\",\n  \"17\": \"no\",\n  \"18\": \"yes\",\n  \"19\": \"no\",\n  \"20\": \"yes\"\n}\n```"
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": name
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{encode_image(img)}",
            }
          }
        ]
      },
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
    )

    try:
        content = completion.choices[0].message.content
        ans = json.loads(content)
        tags = []
        for q_num, answer in ans.items():                 # q_num = "1", answer = "yes"/"no"
            tag_list = map_ans.get(q_num, {}).get(answer.lower(), [])
            tags.extend(tag_list)

        # Deduplicate and keep lowercase
        tags = list(set(map(str.lower, tags)))
        return tags      
    except:
        return ""




In [ ]:
from groq import Groq
import base64, json, time, logging


client = Groq(api_key="")

map_ans = {
   "1": {"yes": ["formal"], "no": []},
   "2": {"yes": ["summer"], "no": []},
   "3": {"yes": ["date", "date-night", "romance"], "no": []},
   "4": {"yes": ["casual"], "no": []},
   "5": {"yes": ["sexy", "hot", "attractive"], "no": []},
   "6": {"yes": ["beach", "pool"], "no": []},
   "7": {"yes": ["bikini", "swimwear", "swim", "ocean", "beach"], "no": []},
   "8": {"yes": ["streetwear"], "no": ["urban"]},
   "9": {"yes": ["sport", "workout", "gym", "fit"], "no": []},
   "10": {"yes": ["party", "cool"], "no": []},
   "11": {"yes": ["modest"], "no": ["conservative"]},
   "12": {"yes": ["trendy"], "no": []},
   "13": {"yes": ["retro", "vintage", "nostalgic"], "no": []},
   "14": {"yes": ["winter"], "no": []},
   "15": {"yes": ["layering"], "no": []},
   "16": {"yes": ["light", "lightweight", "breathable", "breath"], "no": []},
   "17": {"yes": ["business", "work", "office", "professional"], "no": []},
   "18": {"yes": ["minimalist"], "no": ["maximalist", "bold"]},
   "19": {"yes": ["luxury"], "no": []},
   "20": {"yes": ["travel", "comfy"], "no": []},
   "21": {"yes": ["striped"], "no": []},
   "22": {"yes": ["floral"], "no": []},
   "23": {"yes": ["plaid"], "no": []},
   "24": {"yes": ["animal"], "no": []},
   "25": {"yes": ["camo"], "no": []},
   "26": {"yes": ["tiedye"], "no": []},
   "27": {"yes": ["polca-dots"], "no": []},
}


MAX_RETRIES = 3          # how many times to retry on bad JSON
RETRY_SLEEP = 1.5        # seconds between retries
BASE_SYSTEM_PROMPT = "Take name and photo of a product and answer 28 static questions about it. Your task is to analyze the product and determine the most suitable answer for each question.\n\n**Input:**\n\n* A single photo of a product (attachment)\n\n**Task:**\n\n1. Analyze the photo and determine the most suitable answer for each of the 28 questions.\n2. Based on your analysis, assign a answer to each question (e.g. \"yes\", \"no\")\n3. Return the answers as a JSON object with the question number as the key and the assigned tag as the value.\n\nNote: Last question is asking for the product color, answer in standardized fashion color tags, which can be one of the following:\n[\n  \"black\", \"white\", \"gray\", \"grey\", \"red\", \"blue\", \"green\", \"yellow\", \"orange\", \"brown\", \"pink\", \"purple\", \"beige\", \"tan\", \"navy\", \"olive\",\n  \"ivory\", \"cream\", \"taupe\", \"charcoal\", \"khaki\", \"camel\",\n  \"blush\", \"mint\", \"mustard\", \"burgundy\", \"teal\", \"coral\", \"maroon\", \"mauve\", \"sage\", \"lavender\", \"peach\", \"rust\", \"dusty rose\",\n  \"metallic\", \"gold\", \"silver\", \"neon\", \"pastel\", \"ombre\", \"multicolor\", \"colorblock\"\n]\n\n**Questions:**\n\n1. Can this product be worn in a formal Setting? \n2. Can this product be worn in summer weather? \n3. Is this product suitable for a date night? \n4. Is this product good for casual Setting? \n5. Is this product considered Sexy? \n6. Is this item for the beach or poolside? \n7. Is this item from swimwear? is it a bikini? \n8. Does this product lean toward streetwear or urban fashion (yes for streetwear)? \n9. Is this suitable for working out or sports activities? \n10. Does this product look cool enough for a party or club, like does it contain glitter, glow, cleavage?\n11. Does this outfit appear modest or conservative (yes for modest)? \n12. Is this product aligned with current trends? \n13. Does this outfit have a nostalgic or vintage feel? \n14. Is this item suitable for cold weather and winter?\n15. Can this item be layered with other pieces (like jackets)? \n16. Is the fabric light and breathable? \n17. Would you wear this to work or a professional setting? \n18. Is the aesthetic minimalist or maximalist (yes for minimalist)? \n19. Does this product look luxurious or high-end? \n20. Is this item suitable for travel or long wear (yes for travel)? \n21. This product contain a Stripe Pattern?\n22. This product contain a Floral Pattern?\n23. This product contain a Plaid Pattern?\n24. This product contain a Animal Print Pattern?\n25. This product contain a Camouflage (Camo) Pattern?\n26. This product contain a Tie-Dye Pattern?\n27. This product contain a polca dots Pattern?\n28. what color is the product from the standardized fashion color tags?\n\n**JSON Output:**\n```json\n{\n  \"1\": \"no\",\n  \"2\": \"yes\",\n  \"3\": \"no\",\n  \"4\": \"no\",\n  \"5\": \"yes\",\n  \"6\": \"no\",\n  \"7\": \"yes\",\n  \"8\": \"yes\",\n  \"9\": \"no\",\n  \"10\": \"yes\",\n  \"11\": \"yes\",\n  \"12\": \"no\",\n  \"13\": \"yes\",\n  \"15\": \"yes\",\n  \"16\": \"yes\",\n  \"17\": \"no\",\n  \"18\": \"yes\",\n  \"19\": \"no\",\n  \"20\": \"yes\",\n  \"21\": \"yes\",\n  \"22\": \"yes\",\n  \"23\": \"no\",\n  \"24\": \"no\",\n  \"25\": \"yes\",\n  \"26\": \"no\",\n  \"27\": \"no\",\n  \"28\": \"beige\"\n}\n```"

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
  
def groq_call(img_path, name, extra_prompt=""):
    """Single Groq JSON‑mode call; returns content string or raises BadRequestError."""
    return client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {
                "role": "system",
                "content": BASE_SYSTEM_PROMPT + extra_prompt
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": name},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{encode_image(img_path)}"}
                    }
                ]
            }
        ],
        response_format={"type": "json_object"},
        temperature=0.3,               # lower temp → fewer JSON glitches
        max_completion_tokens=256,
        top_p=1,
        stream=False,
    ).choices[0].message.content

def get_tags(img_path, name):
    """Returns list[str] tags derived from Groq JSON; retries on malformed JSON."""
    attempt = 0
    while attempt < MAX_RETRIES:
        try:
            extra = "\n\nONLY return valid JSON." if attempt > 0 else ""
            content = groq_call(img_path, name, extra_prompt=extra)
            ans = json.loads(content)                     # will raise if invalid
            tags = []
            for q, reply in ans.items():
                if q == "28": # color
                    tags.append(reply.lower())
                    continue
                tags.extend(map_ans.get(q, {}).get(reply.lower(), []))
            tags =  list(set(map(str.lower, tags)))       # dedupe + lowercase
            if "urban" in tags and "romance" in tags:
              tags.remove("romance")
            if "maximalist" in tags and "conservative" in tags:
              tags.remove("conservative")
            return tags

        except Exception as e:
            err_str = str(e)
            # Retry only for JSON‑related failures
            if "json_validate_failed" in err_str or "Failed to generate JSON" in err_str:
                logging.warning(f"Bad JSON on try {attempt+1}: {err_str}")
                attempt += 1
                time.sleep(RETRY_SLEEP)
                continue
            else:
                # Some other error (network, auth, quota) – re‑raise
                raise

    logging.error("Failed to get valid JSON after retries.")
    return []   # graceful fallback



In [ ]:
# Vectorize title + desc + image
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

# Load FashionCLIP model and processor from Hugging Face
model_name = "patrickjohncyh/fashion-clip"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

def vectorize(text, img):
    # Load image
    image = Image.open(img).convert("RGB")

    # 1. Get image embedding
    img_inputs = processor(images=image, return_tensors="pt")
    image_emb = model.get_image_features(**img_inputs)

    # 2. Get text embedding
    txt_inputs = processor(text=text, return_tensors="pt", padding=True)
    text_emb = model.get_text_features(**txt_inputs)

    # 3. Normalize both embeddings
    image_emb = torch.nn.functional.normalize(image_emb, p=2, dim=1)
    text_emb = torch.nn.functional.normalize(text_emb, p=2, dim=1)

    # 4. Combine and normalize again
    combined_emb = (image_emb + text_emb) / 2
    combined_emb = torch.nn.functional.normalize(combined_emb, p=2, dim=1)

    return combined_emb[0].cpu().tolist()

In [ ]:
# men
# load the json
import json, time
from pinecone import Pinecone, PodSpec

f = open("men_db.json", 'r')
d = json.load(f)
pc = Pinecone(api_key="")
index = pc.Index("menfit")
SKIP_TO_ID  = None



for i in d:
    if SKIP_TO_ID:
        if not i['id'] == SKIP_TO_ID:
            continue
        else:
            SKIP_TO_ID = None
    text = (f"{i['name']} . {i['description']}")[:75]
    img = i['path']
    i['tags'] = get_tags(img, i['name'])

    print(i['id'], i['tags'])

    # get vector
    vec = vectorize(text, img)

    # save in pinecone
    index.upsert([{
        "id": i['id'],
        "values": vec,
        "metadata": i
    }])
    time.sleep(1)

In [ ]:
# women
# load the json
import json
from pinecone import Pinecone, PodSpec

f = open("women_db.json", 'r')
d = json.load(f)
pc = Pinecone(api_key="")
index = pc.Index("womanfit")



for i in d:
    i['tags'] = get_tags(i['description'])
    text = f"{i['brand']} {i['name']} . {i['description']}"
    img = i['path']

    # get vector
    vec = vectorize(text, img)

    # save in pinecone
    index.upsert([{
        "id": i['id'],
        "values": vec,
        "metadata": i
    }])